# Задача

Делаем анализ, который обсуждали.

*Исходник данных*: продажи за 2 года.

1. На 2016.12.31 получаем инфу по кол-ву заказов на каждого юзера. Подсчитываем сколько пользователей попало в группу с 1 заказом, 2, 3 и т.д.
2. За январь 2017 смотрим, какое кол-во юзеров из п.1 купило в каждой группе и на какую сумму за месяц.(скрипты вам должны помочь)
3. Подсчитываем вероятность перехода в покупающего по каждой группе и ср траты за январь в группе (как на занятии).
4. В идеале сделать тоже самое на февраль, март, и до конца 17 года (чтобы оценить сезонность и рост трат за месяц)

# Подготовка

## Подключение библиотек

In [1]:
import pymysql.cursors
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
import getpass

## Подключение к базе данных

In [2]:
# password_mysql = getpass.getpass(prompt="Введите пароль от MySQL-сервера: ")
password_mysql = "pass123"
mydb = pymysql.connect(host="127.0.0.1",
                             user='root',
                             password=password_mysql, 
                             db='db_analyt_loc',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
 
print ("connect successful!!")

connect successful!!


In [3]:
current_db = "orders_20190822"

In [19]:
# # пример
# pd.read_sql(f"SELECT * FROM {current_db} WHERE price <= 0", con=mydb).head()

,id_o,user_id,price,o_date
0,1401600,272316,-3.5,2016-02-20
1,1615909,926637,0.0,2016-04-13
2,1627392,428359,-14.0,2016-04-15
3,1784326,215957,0.0,2016-05-23
4,1785877,222822,-17.5,2016-05-24


In [13]:
## Второй раз выполнять не надо


# mydb.cursor().execute(f"CREATE TABLE time_user_b SELECT user_id, COUNT(id_o) AS n, DATE('2017-01-01') AS m FROM orders_20190822 AS o WHERE DATE(o_date) < DATE('2017-01-01') GROUP BY user_id")
# mydb.commit()
# print(mydb.cursor())

In [15]:
## Второй раз выполнять не надо

# %%time
# months = ['02','03','04','05','06','07','08','09','10','11','12']
# for month in months:
#     mydb.cursor().execute(f"INSERT INTO time_user_b SELECT user_id, count(id_o) n, DATE('2017-{month}-01') FROM orders_20190822 o WHERE DATE(o_date) < DATE('2017-{month}-01') GROUP BY user_id")
# mydb.commit()

Wall time: 3min


In [4]:
%%time
mydb.cursor().execute(f"CREATE TABLE time_user_ym_s SELECT o.user_id, DATE_FORMAT(o.o_date, '%y%m') AS ym, SUM(o.price) AS s FROM orders_20190822 o GROUP BY o.user_id, DATE_FORMAT(o.o_date, '%y%m')")
mydb.commit()

Wall time: 22.4 s


In [16]:
%%time
df = pd.read_sql(f"SELECT t.n, COUNT(DISTINCT t.user_id), COUNT(s.user_id), SUM(s.s), COUNT(s.user_id)/COUNT(DISTINCT t.user_id) AS p, SUM(s.s)/COUNT(s.user_id) AS s_m, s.ym FROM time_user_b t LEFT JOIN time_user_ym_s s ON t.user_id = s.user_id WHERE t.m = DATE('2017-01-01') AND s.ym = '1701' GROUP BY t.n", con=mydb)

Wall time: 9.7 s


In [17]:
%%time
months = ['02','03','04','05','06','07','08','09','10','11','12']
for month in months:
    print(month)
    df = df.append(pd.read_sql(f"SELECT t.n, COUNT(DISTINCT t.user_id), COUNT(s.user_id), SUM(s.s), COUNT(s.user_id)/COUNT(DISTINCT t.user_id) AS p, SUM(s.s)/COUNT(s.user_id) AS s_m, s.ym FROM time_user_b t LEFT JOIN time_user_ym_s s ON t.user_id = s.user_id WHERE t.m = DATE('2017-{month}-01') AND s.ym = '17{month}' GROUP BY t.n", con=mydb), ignore_index=True)
df.shape

02
03
04
05
06
07
08
09
10
11
12
Wall time: 1min 50s


(1934, 7)

In [18]:
df['ym'].value_counts()

1708    177
1712    176
1710    175
1711    172
1709    170
1705    162
1706    161
1707    160
1704    152
1702    147
1703    143
1701    139
Name: ym, dtype: int64

In [21]:
df.to_excel('les_4_analys.xlsx')

# Закрытие сессии

In [23]:
mydb.close()